In [39]:
import pandas as pd
import pickle
# !pip install -U elecsim
from elecsim import World
from elecsim.market.electricity.market.power_exchange import PowerExchange
from elecsim.agents.demand.multi_day_demand import MultiDayDemand
import elecsim.scenario.scenario_data
from elecsim.plants.plant_costs.estimate_costs.estimate_costs import create_power_plant, create_power_plant_group
from elecsim.role.investment.calculate_npv import CalculateNPV

import array
import numpy
import random
import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [2]:
multi_year_data_scaled = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/elecsim/data/processed/multi_day_data/4_medoids_scaled.csv')
multi_year_data_scaled.head()

,Unnamed: 0,demand_hour,data_type,capacity_factor
0,0,1,load,0.343182
1,1,2,load,0.336312
2,2,3,load,0.308192
3,3,4,load,0.286708
4,4,5,load,0.247604


In [3]:
power_plants = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/elecsim/data//processed/power_plants/uk_power_plants/uk_power_plants.csv')
power_plants.head()

,Unnamed: 0,Unnamed: 0.1,Company,Name,Fuel,Capacity,Start_date,Simplified_Type
0,0,0,AES,Ballylumford B,Gas,540.0,1968.0,Recip_gas
1,1,1,AES,Ballylumford B OCGT,Gas oil,116.0,1968.0,Recip_diesel
2,2,2,AES,Ballylumford C,CCGT,616.0,2003.0,CCGT
3,3,3,AES,Kilroot OCGT,Gas oil,142.0,1981.0,Recip_diesel
4,4,4,AES,Kilroot,Coal,560.0,1981.0,Coal


In [4]:
power_plants['genco_id'] = power_plants.groupby("Company").grouper.group_info[0]
power_plants.head()

,Unnamed: 0,Unnamed: 0.1,Company,Name,Fuel,Capacity,Start_date,Simplified_Type,genco_id
0,0,0,AES,Ballylumford B,Gas,540.0,1968.0,Recip_gas,0
1,1,1,AES,Ballylumford B OCGT,Gas oil,116.0,1968.0,Recip_diesel,0
2,2,2,AES,Ballylumford C,CCGT,616.0,2003.0,CCGT,0
3,3,3,AES,Kilroot OCGT,Gas oil,142.0,1981.0,Recip_diesel,0
4,4,4,AES,Kilroot,Coal,560.0,1981.0,Coal,0


In [5]:
modern_plant_costs = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/elecsim/data/processed/power_plants/power_plant_costs/modern_power_plant_costs/power_plant_costs_with_simplified_type.csv')
modern_plant_costs.head()


,Unnamed: 0,Plant_Type,Plant_Size,Average_Load_Factor,Efficiency,Operating_Period,Constr_1,Constr_2,Constr_3,Constr_4,Constr_5,Constr_6,Constr_7,Constr_8,Constr_Dur,Pre_1,Pre_2,Pre_3,Pre_4,Pre_5,Pre_6,Pre_7,Pre_8,Pre_Dur,Connect_system_cost-Medium _2018,Connect_system_cost-Medium _2020,Connect_system_cost-Medium _2025,Constr_cost-High _2018,Constr_cost-High _2020,Constr_cost-High _2025,Constr_cost-Low _2018,Constr_cost-Low _2020,Constr_cost-Low _2025,Constr_cost-Medium _2018,Constr_cost-Medium _2020,Constr_cost-Medium _2025,Fixed_cost-Medium _2018,Fixed_cost-Medium _2020,Fixed_cost-Medium _2025,Infra_cost-High _2018,Infra_cost-High _2020,Infra_cost-High _2025,Infra_cost-Low _2018,Infra_cost-Low _2020,Infra_cost-Low _2025,Infra_cost-Medium _2018,Infra_cost-Medium _2020,Infra_cost-Medium _2025,Insurance_cost-Medium _2018,Insurance_cost-Medium _2020,Insurance_cost-Medium _2025,Pre_dev_cost-High _2018,Pre_dev_cost-High _2020,Pre_dev_cost-High _2025,Pre_dev_cost-Low _2018,Pre_dev_cost-Low _2020,Pre_dev_cost-Low _2025,Pre_dev_cost-Medium _2018,Pre_dev_cost-Medium _2020,Pre_dev_cost-Medium _2025,Var_cost-Medium _2018,Var_cost-Medium _2020,Var_cost-Medium _2025,Type
0,2,Coal - CCS ASC NH3 FOAK,624.0,0.91,0.32,25.0,0.200,0.200,0.200,0.2,0.2,NaN,NaN,NaN,5,0.200,0.200,0.200,0.200,0.20,NaN,NaN,NaN,5,NaN,NaN,3800.0,NaN,NaN,5500000.0,NaN,NaN,3400000.0,NaN,NaN,4200000.0,NaN,NaN,79600.0,NaN,NaN,15000.0,NaN,NaN,5000.0,NaN,NaN,10000.0,NaN,NaN,19300.0,NaN,NaN,110000.0,NaN,NaN,60000.0,NaN,NaN,70000.0,NaN,NaN,3.0,Coal
1,5,PV 1-5MW building,1.0,0.11,0.00,25.0,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1300.0,1300.0,1200.0,800000.0,800000.0,700000.0,700000.0,600000.0,600000.0,700000.0,700000.0,600000.0,6600.0,6300.0,5900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2600.0,2600.0,2400.0,20000.0,20000.0,20000.0,20000.0,20000.0,20000.0,20000.0,20000.0,20000.0,3.0,3.0,3.0,PV
2,6,EfW,30.0,0.81,0.28,35.0,0.333,0.333,0.333,NaN,NaN,NaN,NaN,NaN,3,0.228,0.228,0.228,0.228,0.09,NaN,NaN,NaN,5,16700.0,16500.0,16100.0,12500000.0,12400000.0,12200000.0,4600000.0,4600000.0,4500000.0,8200000.0,8100000.0,8000000.0,139500.0,137900.0,134800.0,6600.0,6600.0,6600.0,2500.0,2500.0,2500.0,4600.0,4600.0,4600.0,30500.0,30100.0,29500.0,390000.0,390000.0,390000.0,110000.0,110000.0,110000.0,230000.0,230000.0,230000.0,25.0,25.0,24.0,EfW
3,7,EfW CHP,24.0,0.81,0.22,35.0,0.333,0.333,0.333,NaN,NaN,NaN,NaN,NaN,3,0.228,0.228,0.228,0.228,0.09,NaN,NaN,NaN,5,16700.0,16500.0,16100.0,15900000.0,15800000.0,15600000.0,10300000.0,10300000.0,10100000.0,13300000.0,13200000.0,13000000.0,153000.0,151300.0,147900.0,8500.0,8500.0,8500.0,3000.0,3000.0,3000.0,5900.0,5900.0,5900.0,85200.0,84300.0,82400.0,300000.0,300000.0,300000.0,130000.0,130000.0,130000.0,210000.0,210000.0,210000.0,55.0,55.0,53.0,EfW
4,8,AD,2.0,0.79,0.40,20.0,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.667,0.333,NaN,NaN,NaN,NaN,NaN,NaN,2,12900.0,12900.0,12900.0,4900000.0,4900000.0,4900000.0,2900000.0,2900000.0,2900000.0,3700000.0,3700000.0,3700000.0,78100.0,78100.0,78100.0,1000.0,1000.0,1000.0,500.0,500.0,500.0,700.0,700.0,700.0,54200.0,54200.0,54200.0,440000.0,440000.0,440000.0,260000.0,260000.0,260000.0,340000.0,340000.0,340000.0,81.0,81.0,81.0,AD


In [6]:
scenario_2013 = "/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/run/beis_case_study/scenario/reference_scenario_2018.py"

model = World(initialization_year=2018, scenario_file=scenario_2013, market_time_splices=8, data_folder="best_run_beis_comparison", number_of_steps=8*5, fitting_params=[0.001644, 11.04157], highest_demand=63910)


In [7]:
demand = MultiDayDemand(model, 1, elecsim.scenario.scenario_data.multi_year_data)
demand.step()

In [8]:
model.get_gencos()[0].plants.append(create_power_plant_group(name="invested_plant_group", start_date=2019, simplified_type="CCGT", capacity=1400, number_of_plants_to_purchase=2000))




In [9]:
model.get_gencos()[0].plants

[FuelPlant(Ballylumford B, Recip_gas, 540.0, 1968, 0.05, 2.0, 1.0, 15.0, [0.5, 0.5], [1.0], 10000.0, 300000.0, 3400.0, 10000.0, 2.3368378049407688, 1000.0, -28500.0),
 FuelPlant(Ballylumford B OCGT, Recip_diesel, 116.0, 1968, 0.01, 2.0, 1.0, 15.0, [0.5, 0.5], [1.0], 10000.0, 300000.0, 2200.0, 10000.0, 1.7492371042815296, 1000.0, -28000.0),
 FuelPlant(Ballylumford C, CCGT, 616.0, 2003, 0.93, 3.0, 3.0, 25.0, [0.435, 0.435, 0.13], [0.4, 0.4, 0.2], 97688.08244521318, 1564195.814608266, 36354.204610678746, 54218.863251291485, 6.577049344698402, 6511.879666073973, 8418.181519598405),
 FuelPlant(Kilroot OCGT, Recip_diesel, 142.0, 1981, 0.01, 2.0, 1.0, 15.0, [0.5, 0.5], [1.0], 10000.0, 300000.0, 2200.0, 10000.0, 1.1905760379068093, 1000.0, -28000.0),
 FuelPlant(Kilroot, Coal, 560.0, 1981, 0.91, 5.0, 5.0, 25.0, [0.2, 0.2, 0.2, 0.2, 0.2], [0.2, 0.2, 0.2, 0.2, 0.2], 192767.11861957662, 11566027.121606465, 27538.15980402428, 219203.7520392322, 6.443852464068768, 53148.648421338505, 10464.500727213

In [10]:
model.step()

2018:

KeyboardInterrupt: 

In [11]:
model.operate_constructed_plants(2050)

In [12]:
PowerExchange(model).tender_bids(demand.segment_hours, demand.segment_consumption)

,accepted_price,day,segment_demand,segment_hour,year
0,37.561456,1,53460.929489,26.0,2018
1,37.561456,1,53323.976240,52.0,2018
2,37.561456,1,49719.296516,78.0,2018
3,37.561456,1,44670.462348,104.0,2018
4,37.561456,1,44634.468225,130.0,2018
5,37.561456,1,41728.601229,156.0,2018
6,37.561456,1,41362.514662,182.0,2018
7,37.561456,1,41334.421688,208.0,2018
8,37.561456,1,41135.137154,234.0,2018
9,37.561456,1,40786.608695,260.0,2018


In [60]:
potential_plants_to_invest = ['CCGT H Class', 'CCGT F Class', 'CCGT CHP mode', 'Coal - CCS ASC NH3 FOAK', 'Coal - CCS ASC Partial FOAK', 'Coal - CCS ASC FOAK', 'Nuclear - PWR FOAK', 'Onshore UK>5MW', 'Offshore R2', 'Offshore R3', 'PV>5MW', 'Recip Gas 2000 hr', 'RECIP GAS 500 hrs', 'Recip Diesel 2000 hr', 'Recip Diesel 500 hrs', 'Recip Diesel 90 hrs']

def evalMin(model):
    print("evaluating")
    individual = [model.one, model.two, model.three, model.four, model.five, model.six, model.seven, model.eight, model.nine, model.ten, model.eleven, model.twelve, model.thirteen, model.fourteen, model.fifteen, model.sixteen]
    model = World(initialization_year=2018, scenario_file=scenario_2013, market_time_splices=8, data_folder="best_run_beis_comparison", number_of_steps=8*5, fitting_params=[0.001644, 11.04157], highest_demand=63910)
    demand = MultiDayDemand(model, 1, elecsim.scenario.scenario_data.multi_year_data)

        
    for number_of_plants, plant_string in zip(individual, potential_plants_to_invest):
        plant = elecsim.scenario.scenario_data.modern_plant_costs[elecsim.scenario.scenario_data.modern_plant_costs.Plant_Type.str.contains(plant_string)]
        if pyo.value(number_of_plants)>=1:
            plant_group = create_power_plant_group("plant_RL_invested", 2026, plant.Type.values[0], plant.Plant_Size.values[0], pyo.value(number_of_plants))
            model.get_gencos()[0].plants.append(plant_group)
    
    model.operate_constructed_plants(2050)
    pdc = []
    for _ in range(8):
        demand.step()
        electricity_prices = PowerExchange(model).tender_bids(demand.segment_hours, demand.segment_consumption)
        pdc.append(electricity_prices)
    pdc_df = pd.concat(pdc)
    
    total_npv = 0
    for genco in model.get_gencos():
        for plant in genco.plants:
            npv = CalculateNPV(model, 0, 10, pdc_df).calculate_npv(plant.plant_type, plant.capacity_mw)
            if npv > float("-inf"):
                total_npv += npv
            else:
                total_npv += -10000
    
    print("total_npv: {}".format(total_npv))
    return total_npv
#     return sum(individual),

In [28]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 100)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(potential_plants_to_invest))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalMin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(64)
    
    pop = toolbox.population(n=30)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=7, 
                                   stats=stats, halloffame=hof, verbose=True)
    best_ind = tools.selBest(pop, 1)[0]
    print(best_ind)
    return pop, log, hof

if __name__ == "__main__":
    main()


/Users/b1017579/.pyenv/versions/3.6.6/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/b1017579/.pyenv/versions/3.6.6/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


evaluating
total_npv: -3503290.8661027816
evaluating
total_npv: -3432178.172454269
evaluating
total_npv: -3382697.0212933742
evaluating
total_npv: -3463694.1517993887
evaluating
total_npv: -3476429.3507455913
evaluating
total_npv: -3469954.7979418575
evaluating
total_npv: -4605778.903085935
evaluating
total_npv: -3296743.310997024
evaluating
total_npv: -4486667.297944382
evaluating
total_npv: -3481332.5692599616
evaluating
total_npv: -3585691.5367428795
evaluating
total_npv: -3528057.9721340067
evaluating
total_npv: -3538010.614659533
evaluating
total_npv: -4261334.060035193
evaluating
total_npv: -4207238.545643022
evaluating
total_npv: -3653430.0437142183
evaluating
total_npv: -3565093.5205017407
evaluating
total_npv: -3445911.501893685
evaluating
total_npv: -3569885.2833498446
evaluating
total_npv: -3570625.5051438166
evaluating
total_npv: -3657796.016170298
evaluating
total_npv: -3565450.9645424327
evaluating
total_npv: -4215110.885780787
evaluating
total_npv: -3283585.5338356276
ev

In [ ]:
[82, 0, 0, 0, 0, 29, 3, 15, 47, 0, 4, 61, 25, 0, 41, 54]

['CCGT H Class' - 82
 'CCGT F Class',- 0 
 'CCGT CHP mode', - 0
 'Coal - CCS ASC NH3 FOAK', - 0
 'Coal - CCS ASC Partial FOAK', - 0 
 'Coal - CCS ASC FOAK', - 29
 'Nuclear - PWR FOAK', - 3
 'Onshore UK>5MW', - 15
 'Offshore R2',  - 47
 'Offshore R3', - 0
 'PV>5MW', - 4
 'Recip Gas 2000 hr', - 61
 'RECIP GAS 500 hrs', - 25
 'Recip Diesel 2000 hr', - 0
 'Recip Diesel 500 hrs', - 41
 'Recip Diesel 90 hrs'] - 54

## Pyomo Optimisation 

In [12]:
import pyomo.environ as pyo


In [61]:
# model = pyo.ConcreteModel()

# model_nVars = pyo.Param(initialize=4)

# model.m = pyo.Param(model_nVars)#, within=pyo.NonNegativeIntegers)

# model_j = pyo.RangeSet(1, model.m)

# m_w = pyo.Var(model_j, domain=pyo.NonNegativeReals)

# m_objective = pyo.Objective(expr=evalMin(m_w), sense=pyo.maximize)

# pyo.SolverFactory('glpk').solve(model)

model = pyo.ConcreteModel()

model.one = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=82)
model.two = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=0)
model.three = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=0)
model.four = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=0)
model.five = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=0)
model.six = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=29)
model.seven = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=3)
model.eight = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=15)
model.nine = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=47)
model.ten = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=0)
model.eleven = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=4)
model.twelve = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=61)
model.thirteen = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=25)
model.fourteen = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=0)
model.fifteen = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=41)
model.sixteen = pyo.Var(bounds=(0,1000), within=pyo.NonNegativeIntegers, initialize=54)

model.Objective = pyo.Objective(expr=evalMin(model), sense=pyo.maximize)
# model.cuts = pyo.ConstraintList()
opt = pyo.SolverFactory('glpk')
opt.solve(model)

evaluating


total_npv: -3102973.976282306
    solver failure.


{'Problem': [{'Name': 'unknown', 'Lower bound': -3102973.97628231, 'Upper bound': -3102973.97628231, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 1, 'Number of nonzeros': 1, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.028257131576538086}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}